# Otobüs Hareket Saatleri ve Durakları Verisetleri Analizi

Bu çalışmada İzmir’de 757 ve 619 numaralı otobüs hatları için saat–gün bazında sefer yoğunluğunu çıkardım. Hareket saatlerini okuyup TARIFE_ID’yi gün tipine çevirdim, gidiş/dönüş saatlerini tek kolonda birleştirip gün×saat kırılımında sefer sayılarını saydım ve ısı haritalarını ürettim. Duraklar verisinden iki hattın duraklarını harita üzerinde gösterdim ve Ulukent Aktarma Merkezi ile Bakırçay Üniversitesi için 300 m içinde durak olup olmadığını mesafe (haversine) hesabıyla doğruladım. Ek olarak, tüm hatlar için en çok sefer yapanlar sıralamasını çıkardım. (İleride K-Means ile hat profili kümeleme eklenebilir.)

In [2]:
import os
os.environ["OMP_NUM_THREADS"] = "2"  

import os, random, numpy as np
os.environ["OMP_NUM_THREADS"] = "2"  # Windows+MKL uyarısını bastırmak için
random.seed(42); np.random.seed(42)


### Veri Okuma ve Kurulum

In [4]:

# Grafiklerin inline görünmesi
%matplotlib inline

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dosyalarımızı tanımlayalım
DESKTOP = Path.home() / "Desktop"
p_hareket = DESKTOP / "eshot-otobus-hareketsaatleri.csv"
p_durak   = DESKTOP / "eshot-otobus-duraklari.csv"

# CSV'leri okuyalım
hareket = pd.read_csv(p_hareket, sep=";", encoding="utf-8")
duraklar = pd.read_csv(p_durak, sep=";", encoding="utf-8")

# Tip ve hızlı kontrol yapalım
hareket["HAT_NO"] = pd.to_numeric(hareket["HAT_NO"], errors="coerce")
print("hareket:", hareket.shape, "| duraklar:", duraklar.shape)
display(hareket.head(3))
display(duraklar.head(3))


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cemre\\Desktop\\eshot-otobus-hareketsaatleri.csv'

### Seçilen Hatlar

In [ ]:
# TARIFE_ID → Gün adı olarak gruplayalım
tarife_map = {1: "Hafta içi", 2: "Cumartesi", 3: "Pazar"}
hareket["GUN"] = hareket["TARIFE_ID"].map(tarife_map)

# Gidiş/Dönüş saatlerini tek kolonda toplayalım 
long = pd.melt(
    hareket,
    id_vars=["HAT_NO", "TARIFE_ID", "GUN"],
    value_vars=["GIDIS_SAATI", "DONUS_SAATI"],
    var_name="YON",
    value_name="SAAT_STR",
)

# Geçersiz saatleri atalım ve 0–23 saat değerini çıkar
long = long[long["SAAT_STR"].str.match(r"^\d{2}:\d{2}$", na=False)].copy()
long["SAAT"] = pd.to_datetime(long["SAAT_STR"], format="%H:%M").dt.hour


# Okul için kullandığımız hatları seçmek istedim.Fakat 812 sonradan eklenen bir hat olduğu için verisetine dahil edilmemiş. Bu yüzden 619 ve 757 ile çalışmaya devam ediyorum.
print("long:", long.shape, "| 757 var mı?", (long["HAT_NO"]==757).any(), "| 619 var mı?", (long["HAT_NO"]==619).any(), "| 812 var mı?", (long["HAT_NO"]==812).any())


### Sefer Sıklığı Isı Haritaları

In [ ]:

def plot_heatmap(df_long: pd.DataFrame, hat_no: int, save_dir=DESKTOP, vmin=0, vmax=None):
    gun_sira = ["Hafta içi", "Cumartesi", "Pazar"]
    df_hat = df_long[df_long["HAT_NO"] == hat_no].copy()
    assert not df_hat.empty, f"HAT {hat_no}: veri bulunamadı."

    pivot = pd.crosstab(df_hat["GUN"], df_hat["SAAT"]).reindex(gun_sira).fillna(0)
    pivot = pivot.reindex(columns=list(range(24)), fill_value=0)  # 0–23 eksiksiz

    # vmax dışarıdan verilmemişse bu hattın tavanını kullan
    vmax = pivot.values.max() if vmax is None else vmax

    fig, ax = plt.subplots(figsize=(10,4))
    im = ax.imshow(pivot.values, aspect="auto", vmin=vmin, vmax=vmax)
    ax.set_title(f"HAT {hat_no} - Saat–Gün Isı Haritası (Sefer Yoğunluğu)")
    ax.set_xlabel("Saat (0–23)")
    ax.set_ylabel("Gün")
    ax.set_yticks(range(len(gun_sira)))
    ax.set_yticklabels(gun_sira)
    fig.colorbar(im, ax=ax, label="Sefer sayısı")
    fig.tight_layout()

    out_png = (save_dir / f"heatmap_hat_{hat_no}.png").as_posix()
    fig.savefig(out_png, dpi=150)
    print("Kaydedildi:", out_png)

    plt.show()
    plt.close(fig)

# --- ORTAK vmax HESABI ve ÇİZİM 
subset = long[ long["HAT_NO"].isin([757, 619]) ]
cnt = (subset.groupby(["HAT_NO","GUN","SAAT"]).size().rename("cnt"))
global_vmax = int(cnt.max()) if not cnt.empty else None



for h in [757, 619]:
    plot_heatmap(long, h, vmin=0, vmax=global_vmax)



ISI HARİTALARI YORUMU

757: Hafta içi 06–08’de keskin bir sabah zirvesi var; 08 sonrası gün boyu yüksek ve sürekli bir plato görülüyor. Hafta sonu belirgin şekilde seyrekleşiyor, Pazar en düşük.
619: Hafta içi 06’dan sonra hızla yükselip gün boyunca yüksek kalıyor; akşama kadar hizmet güçlü. Hafta sonu düşüyor, Pazar en düşük.
Karşılaştırma: İki hat da hafta içi daha sık; 757’de zirve daha keskin, 619’da yoğunluk daha dengeli ve gün boyu yayılmış. (Haritalar aynı renk ölçeğiyle çizildi, bu yüzden farklar doğrudan karşılaştırılabilir.)


### Durakların Haritada Gösterimi

In [ ]:
# HARİTA BLOĞU 
import pandas as pd, folium
from pathlib import Path
from IPython.display import display, IFrame

assert 'duraklar' in globals(), "Önce duraklar CSV'sini oku: duraklar = pd.read_csv(..., sep=';')"

def stops_for(hat_no: int, df: pd.DataFrame) -> pd.DataFrame:
    s = df["DURAKTAN_GECEN_HATLAR"].astype(str)
    def has_exact(cell):
        raw = str(cell)
        for sep in [";", ",", "/", "\\", "|", "-", " "]:
            raw = raw.replace(sep, "|")
        toks = [t.strip() for t in raw.split("|") if t.strip()]
        return str(hat_no) in toks
    mask = s.apply(has_exact) | s.str.contains(fr"\b{hat_no}\b", regex=True, na=False)
    out = df[mask].copy()
    out["ENLEM"]  = pd.to_numeric(out["ENLEM"], errors="coerce")
    out["BOYLAM"] = pd.to_numeric(out["BOYLAM"], errors="coerce")
    return out.dropna(subset=["ENLEM","BOYLAM"]).drop_duplicates(subset=["DURAK_ID"]).reset_index(drop=True)

# Veriler
d757 = stops_for(757, duraklar)
d619 = stops_for(619, duraklar)
print(f"Durak sayısı -> 757: {len(d757)} | 619: {len(d619)}")

# Merkez (fallback'lı)
all_pts = pd.concat(
    [d757[["ENLEM","BOYLAM"]], d619[["ENLEM","BOYLAM"]]],
    ignore_index=True
)
if not all_pts.empty and all_pts["ENLEM"].notna().any() and all_pts["BOYLAM"].notna().any():
    center = [all_pts["ENLEM"].mean(), all_pts["BOYLAM"].mean()]
else:
    # İzmir genel fallback veya bildiğin bir nokta
    center = [38.42, 27.14]  # İzmir merkez civarı
    print("Uyarı: Merkez için yedek konum kullanıldı (veri boş olabilir).")

# Harita
m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")

def add_layer(df, name, color):
    fg = folium.FeatureGroup(name=name, show=True)
    for _, r in df.iterrows():
        folium.CircleMarker(
            [float(r["ENLEM"]), float(r["BOYLAM"])],
            radius=3, color=color, fill=True, fill_color=color, fill_opacity=0.9,
            tooltip=folium.Tooltip(f"{r['DURAK_ADI']}", sticky=True)
        ).add_to(fg)
    fg.add_to(m)

add_layer(d757, f"HAT 757 (n={len(d757)})", "#e74c3c")
add_layer(d619, f"HAT 619 (n={len(d619)})", "#1f77b4")
folium.LayerControl(collapsed=False).add_to(m)


display(m)







In [ ]:
# Bilinen noktalardan 757 & 619 geçiyor mu? 
import numpy as np, pandas as pd

assert 'duraklar' in globals(), "Önce duraklar CSV'sini oku: duraklar = pd.read_csv(..., sep=';')"

# 1) Bilinen noktalar ve eşik
known_points = [
    {"name": "Ulukent Aktarma Merkezi", "lat": 38.547269, "lon": 27.035109},
    {"name": "Bakırçay Üniversitesi",   "lat": 38.58229,  "lon": 26.964101},
]
radius_m = 300
hats = [757, 619]

# 2) Hattan geçen durakları eksiksiz çekelim 
def stops_for(hat_no: int, df: pd.DataFrame) -> pd.DataFrame:
    s = df["DURAKTAN_GECEN_HATLAR"].astype(str)
    def has_exact(cell):
        raw = str(cell)
        for sep in [";", ",", "/", "\\", "|", "-", " "]:
            raw = raw.replace(sep, "|")
        toks = [t.strip() for t in raw.split("|") if t.strip()]
        return str(hat_no) in toks
    mask = s.apply(has_exact) | s.str.contains(fr"\b{hat_no}\b", regex=True, na=False)
    out = df[mask].copy()
    out["ENLEM"]  = pd.to_numeric(out["ENLEM"], errors="coerce")
    out["BOYLAM"] = pd.to_numeric(out["BOYLAM"], errors="coerce")
    return out.dropna(subset=["ENLEM","BOYLAM"]).drop_duplicates(subset=["DURAK_ID"]).reset_index(drop=True)

# 3) En yakın durak (vektörize haversine)
def nearest(df, lat, lon):
    lat1, lon1 = np.radians([lat, lon])
    xy = np.radians(df[['ENLEM','BOYLAM']].values)
    dlat, dlon = xy[:,0]-lat1, xy[:,1]-lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(xy[:,0])*np.sin(dlon/2)**2
    dist_m = (6371*2*np.arcsin(np.sqrt(a)))*1000
    i = int(dist_m.argmin())
    return df.iloc[i], float(dist_m[i])

# 4) Tabloyu oluşturalım
dsets = {h: stops_for(h, duraklar) for h in hats}
rows=[]
for p in known_points:
    for h in hats:
        df = dsets[h]
        if df.empty:
            rows.append({"NOKTA":p["name"], "HAT":h, "ESIK_m":radius_m,
                         "GECIYOR_MU":False, "MESAFE_m":None, "EN_YAKIN_DURAK":None, "DURAK_ID":None})
            continue
        near, d = nearest(df, p["lat"], p["lon"])
        rows.append({"NOKTA":p["name"], "HAT":h, "ESIK_m":radius_m,
                     "GECIYOR_MU": d<=radius_m, "MESAFE_m":round(d,1),
                     "EN_YAKIN_DURAK": near["DURAK_ADI"], "DURAK_ID": near["DURAK_ID"]})

res = pd.DataFrame(rows).sort_values(["NOKTA","HAT"]).reset_index(drop=True)
print(res.to_string(index=False))


for p in known_points:
    passed = [str(h) for h in hats if res[(res["NOKTA"]==p["name"]) & (res["HAT"]==h)]["GECIYOR_MU"].iloc[0]]
    print(f"- {p['name']}: ≤{radius_m} m içinde geçen hatlar → {', '.join(passed) if passed else 'yok'}")





### En Çok Sefer Yapan Hatlar

Bu bölümde `long` tablosundan hareketle (her satır = 1 sefer), hat bazında **toplam sefer sayısını** hesaplayıp sıralıyorum. 
Önce genel sıralama, sonra gün tipine göre (Hafta içi / Cumartesi / Pazar) ayrı  grafiklerini oluşturalım

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

# long hazır değilse hızlıca üret (hareket değişkeni varsa)
if 'long' not in globals():
    assert 'hareket' in globals(), "Önce veri hazırlık hücrelerini çalıştır (hareket/long oluşturulmalı)."
    tarife_map = {1: "Hafta içi", 2: "Cumartesi", 3: "Pazar"}
    hareket["HAT_NO"] = pd.to_numeric(hareket["HAT_NO"], errors="coerce")
    hareket["GUN"] = hareket["TARIFE_ID"].map(tarife_map)
    long = pd.melt(hareket, id_vars=["HAT_NO","TARIFE_ID","GUN"],
                   value_vars=["GIDIS_SAATI","DONUS_SAATI"],
                   var_name="YON", value_name="SAAT_STR")
    long = long[long["SAAT_STR"].str.match(r"^\d{2}:\d{2}$", na=False)].copy()
    long["SAAT"] = pd.to_datetime(long["SAAT_STR"], format="%H:%M").dt.hour

# ---- Genel sıralama
overall = (long.groupby("HAT_NO").size()
           .rename("SEFER_SAYISI").reset_index()
           .sort_values("SEFER_SAYISI", ascending=False).reset_index(drop=True))
overall["SIRA"] = np.arange(1, len(overall)+1)
display(overall.head(30))  # Top 30 tablo

# CSV olarak kaydet
out_dir = Path(".")
overall_csv = out_dir / "top_hats_overall.csv"
overall.to_csv(overall_csv, index=False, encoding="utf-8")
print("Kaydedildi:", overall_csv.as_posix())

# ---- Top 10 bar (tek grafik)
top10 = overall.head(10).iloc[::-1]  # y ekseninde büyükten küçüğe
plt.figure(figsize=(8,5))
plt.barh(top10["HAT_NO"].astype(str), top10["SEFER_SAYISI"])
plt.title("En Çok Sefer Yapan 10 Hat (Toplam)")
plt.xlabel("Sefer Sayısı (adet)"); plt.ylabel("Hat No")
plt.tight_layout(); plt.show()

# İlgilendiğimiz hatlar nerede?
for h in [757, 619]:
    row = overall.loc[overall["HAT_NO"]==h, ["SIRA","SEFER_SAYISI"]]
    if not row.empty:
        print(f"Hat {h}: Sıra {int(row.iloc[0,0])}, Sefer {int(row.iloc[0,1])}")
    else:
        print(f"Hat {h}: veri setinde yok.")



In [ ]:
# Gün tipine göre Top 10 (her biri ayrı grafik)
by_day = (long.groupby(["HAT_NO","GUN"]).size()
          .rename("SEFER_SAYISI").reset_index())

for gun in ["Hafta içi","Cumartesi","Pazar"]:
    df = (by_day[by_day["GUN"]==gun]
          .sort_values("SEFER_SAYISI", ascending=False).head(10).iloc[::-1])
    if df.empty:
        continue
    plt.figure(figsize=(8,5))
    plt.barh(df["HAT_NO"].astype(str), df["SEFER_SAYISI"])
    plt.title(f"Top 10 Hat – {gun}")
    plt.xlabel("Sefer Sayısı (adet)"); plt.ylabel("Hat No")
    plt.tight_layout(); plt.show()


### Sonuç

Bu çalışmada 757 ve 619 hatları için veriyi hazırlayıp gün×saat düzeyinde sefer yoğunluğu çıkardım. Isı haritaları, 757’de hafta içi 06–08 bandında keskin bir sabah zirvesi olduğunu; 619’da ise yoğunluğun gün boyu daha dengeli seyrettiğini ve iki hatta da hafta sonu belirgin düşüş yaşandığını gösterdi. Durak verisi ile yaptığım konumsal doğrulamada Ulukent Aktarma Merkezi ve Bakırçay Üniversitesi için ≤300 m içinde durak tespit ettim (erişim doğrulandı). Ek olarak “en çok sefer yapanlar” sıralaması genel tabloyu destekledi.

In [12]:
# Python ile platformdan bağımsız:
import os
base = "izmir-bus-service-analysis"
for d in ["notebooks", "outputs", "data"]:
    os.makedirs(os.path.join(base, d), exist_ok=True)
print("Klasörler hazır:", os.listdir(base))



Klasörler hazır: ['data', 'notebooks', 'outputs']


In [14]:
from pathlib import Path
base = Path("izmir-bus-service-analysis")

(base/"README.md").write_text(
"# İzmir Otobüs Sefer Yoğunluğu – 757 & 619\n\n"
"Bu repo, 757 ve 619 için gün×saat sefer yoğunluğu, durakların harita üzerinde gösterimi "
"(≤300 m erişim doğrulaması) ve opsiyonel K-Means kümelemeyi içerir.\n\n"
"## Veri Setleri\n"
"- eshot-otobus-hareketsaatleri.csv (ayraç `;`, TARIFE_ID→{Hafta içi, Cumartesi, Pazar} varsayımı)\n"
"- eshot-otobus-duraklari.csv (ENLEM/BOYLAM WGS84, DURAKTAN_GECEN_HATLAR token’lı)\n"
"> Not: Büyük/lisanslı verileri repo dışı tutuyorum; notebook `data/` altından okur.\n\n"
"## Çalıştırma\n"
"```bash\n"
"python -m venv .venv && source .venv/bin/activate  # Windows: .venv\\Scripts\\activate\n"
"pip install -r requirements.txt\n"
"jupyter lab\n"
"```\n"
, encoding="utf-8")

(base/".gitignore").write_text(
"data/*.csv\n.venv/\n__pycache__/\n.ipynb_checkpoints/\n.DS_Store\n", encoding="utf-8")

(base/"requirements.txt").write_text(
"pandas\nnumpy\nmatplotlib\nfolium\nscikit-learn\nthreadpoolctl\njupyterlab\n", encoding="utf-8")

print("Dosyalar yazıldı.")


Dosyalar yazıldı.
